In [1]:
# Import the necessary libraries
import warnings
import openai
import os
import pandas as pd
import numpy as np

# Ignore warnings related to deprecated features
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
# Get the API key and resource endpoint from environment variables
API_KEY = os.getenv("AZURE_OPENAI_API_KEY") 
RESOURCE_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT") 

# Set the API type and key in the openai library
openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
openai.api_version = "2022-12-01"

Possible parameters in a Completition request:

#### model string required

ID of the model to use. You can use the List models API to see all of your available models, or see our Model overview for descriptions of them.

#### prompt (string or array) Optional
Defaults to <|endoftext|>

The prompt(s) to generate completions for, encoded as a string, array of strings, array of tokens, or array of token arrays.

Note that <|endoftext|> is the document separator that the model sees during training, so if a prompt is not specified the model will generate as if from the beginning of a new document.

#### temperature (number) Optional
Defaults to 1

What sampling temperature to use, between 0 and 2. Higher values like 0.8 will make the output more random, while lower values like 0.2 will make it more focused and deterministic.

We generally recommend altering this or top_p but not both.

#### max_tokens (integer) Optional
Defaults to 16

The maximum number of tokens to generate in the completion.

The token count of your prompt plus max_tokens cannot exceed the model's context length. Most models have a context length of 2048 tokens (except for the newest models, which support 4096).

#### stop (string or array) Optional
Defaults to null

Up to 4 sequences where the API will stop generating further tokens. The returned text will not contain the stop sequence.

#### n (integer) Optional
Defaults to 1

How many completions to generate for each prompt.

Note: Because this parameter generates many completions, it can quickly consume your token quota. Use carefully and ensure that you have reasonable settings for max_tokens and stop.

#### suffix (string) Optional
Defaults to null

The suffix that comes after a completion of inserted text.

#### top_p (number) Optional
Defaults to 1

An alternative to sampling with temperature, called nucleus sampling, where the model considers the results of the tokens with top_p probability mass. So 0.1 means only the tokens comprising the top 10% probability mass are considered.

We generally recommend altering this or temperature but not both.

#### stream (boolean) Optional
Defaults to false

Whether to stream back partial progress. If set, tokens will be sent as data-only server-sent events as they become available, with the stream terminated by a data: [DONE] message.

#### logprobs (integer) Optional
Defaults to null

Include the log probabilities on the logprobs most likely tokens, as well the chosen tokens. For example, if logprobs is 5, the API will return a list of the 5 most likely tokens. The API will always return the logprob of the sampled token, so there may be up to logprobs+1 elements in the response.

The maximum value for logprobs is 5. If you need more than this, please contact us through our Help center and describe your use case.

#### echo (boolean) Optional
Defaults to false

Echo back the prompt in addition to the completion

#### presence_penalty (number) Optional
Defaults to 0

Number between -2.0 and 2.0. Positive values penalize new tokens based on whether they appear in the text so far, increasing the model's likelihood to talk about new topics.

See more information about frequency and presence penalties.

#### frequency_penalty (number) Optional
Defaults to 0

Number between -2.0 and 2.0. Positive values penalize new tokens based on their existing frequency in the text so far, decreasing the model's likelihood to repeat the same line verbatim.

See more information about frequency and presence penalties.

#### best_of (integer) Optional
Defaults to 1

Generates best_of completions server-side and returns the "best" (the one with the highest log probability per token). Results cannot be streamed.

When used with n, best_of controls the number of candidate completions and n specifies how many to return – best_of must be greater than n.

Note: Because this parameter generates many completions, it can quickly consume your token quota. Use carefully and ensure that you have reasonable settings for max_tokens and stop.

#### logit_bias (map) Optional
Defaults to null

Modify the likelihood of specified tokens appearing in the completion.

Accepts a json object that maps tokens (specified by their token ID in the GPT tokenizer) to an associated bias value from -100 to 100. You can use this tokenizer tool (which works for both GPT-2 and GPT-3) to convert text to token IDs. Mathematically, the bias is added to the logits generated by the model prior to sampling. The exact effect will vary per model, but values between -1 and 1 should decrease or increase likelihood of selection; values like -100 or 100 should result in a ban or exclusive selection of the relevant token.

As an example, you can pass {"50256": -100} to prevent the <|endoftext|> token from being generated.

#### user (string) Optional
A unique identifier representing your end-user, which can help OpenAI to monitor and detect abuse. Learn more.

# Temperature

Setting the API temperature to 0, or close to zero (such as 0.1 or 0.2) tends to give better results in most cases. Unlike GPT-3 models, where a higher temperature can provide useful creative and random results, higher temperatures with Codex models may give you really random or erratic responses.

### High Temperature example

In [35]:
COMPLETIONS_MODEL = "code-davinci-002"
prompt = "# read a CSV file using PySpark and print the schema\ndef "

# Set the parameter for the completions API call
COMPLETIONS_API_PARAMS = {
    "engine": COMPLETIONS_MODEL,
    "prompt": prompt,
    "max_tokens": 100,
    "temperature": 1,
    "stop": "#"
}

response = openai.Completion.create(
                **COMPLETIONS_API_PARAMS
            )

In [36]:
print("Prompt:\n" + prompt + "\n")
print("Response:\n" + response["choices"][0]["text"])

Prompt:
# read a CSV file using PySpark and print the schema
def 

Response:
read_and_print_schema():
    df = spark.read.format("csv")\
        .option("header", "true")\
        .option("mode", "FAILFAST")\
        .option("inferSchema", "true")\
        .load("../data/Amazon_Unlocked_Mobile.csv")
    df.printSchema()
    return df





### Low Temperature example

In [33]:
COMPLETIONS_MODEL = "code-davinci-002"
prompt = "# read a CSV file using PySpark and print the schema\ndef "

# Set the parameter for the completions API call
COMPLETIONS_API_PARAMS = {
    "engine": COMPLETIONS_MODEL,
    "prompt": prompt,
    "max_tokens": 100,
    "temperature": 0,
    "stop": "#"
}

response = openai.Completion.create(
                **COMPLETIONS_API_PARAMS
            )

In [34]:
print("Prompt:\n" + prompt + "\n")
print("Response:\n" + response["choices"][0]["text"])

Prompt:
# read a CSV file using PySpark and print the schema
def 

Response:
read_csv_file(spark, file_name):
    df = spark.read.csv(file_name, header=True, inferSchema=True)
    df.printSchema()
    return df




# Max Tokens and Stop

Requesting longer completions in Codex can lead to imprecise answers and repetition. Limit the size of the query by reducing max_tokens and setting stop tokens.

### High max_tokens without stop example

In [4]:
COMPLETIONS_MODEL = "code-davinci-002"
prompt = "# read a CSV file using PySpark and print the schema\ndef "

# Set the parameter for the completions API call
COMPLETIONS_API_PARAMS = {
    "engine": COMPLETIONS_MODEL,
    "prompt": prompt,
    "max_tokens": 2000,
    "temperature": 0
}

response = openai.Completion.create(
                **COMPLETIONS_API_PARAMS
            )

In [5]:
print("Prompt:\n" + prompt + "\n")
print("Response:\n" + response["choices"][0]["text"])

Prompt:
# read a CSV file using PySpark and print the schema
def 

Response:
read_csv_file(spark, file_name):
    df = spark.read.csv(file_name, header=True, inferSchema=True)
    df.printSchema()
    return df

# read a JSON file using PySpark and print the schema
def read_json_file(spark, file_name):
    df = spark.read.json(file_name)
    df.printSchema()
    return df

# read a Parquet file using PySpark and print the schema
def read_parquet_file(spark, file_name):
    df = spark.read.parquet(file_name)
    df.printSchema()
    return df

# read a ORC file using PySpark and print the schema
def read_orc_file(spark, file_name):
    df = spark.read.orc(file_name)
    df.printSchema()
    return df

# read a Avro file using PySpark and print the schema
def read_avro_file(spark, file_name):
    df = spark.read.format("com.databricks.spark.avro").load(file_name)
    df.printSchema()
    return df

# read a file using PySpark and print the schema
def read_file(spark, file_name, file_type

### Low max_tokens without stop example

In [11]:
COMPLETIONS_MODEL = "code-davinci-002"
prompt = "# read a CSV file using PySpark and print the schema\ndef "

# Set the parameter for the completions API call
COMPLETIONS_API_PARAMS = {
    "engine": COMPLETIONS_MODEL,
    "prompt": prompt,
    "max_tokens": 200,
    "temperature": 0,
    "stop": ""
}

response = openai.Completion.create(
                **COMPLETIONS_API_PARAMS
            )

In [12]:
print("Prompt:\n" + prompt + "\n")
print("Response:\n" + response["choices"][0]["text"])

Prompt:
# read a CSV file using PySpark and print the schema
def 

Response:
read_csv_file(spark, file_name):
    df = spark.read.csv(file_name, header=True, inferSchema=True)
    df.printSchema()
    return df

# read a JSON file using PySpark and print the schema
def read_json_file(spark, file_name):
    df = spark.read.json(file_name)
    df.printSchema()
    return df

# read a Parquet file using PySpark and print the schema
def read_parquet_file(spark, file_name):
    df = spark.read.parquet(file_name)
    df.printSchema()
    return df

# read a ORC file using PySpark and print the schema
def read_orc_file(spark, file_name):
    df = spark


### Low max_tokens with custom stop "#" example

In [8]:
COMPLETIONS_MODEL = "code-davinci-002"
prompt = "# read a CSV file using PySpark and print the schema\ndef "

# Set the parameter for the completions API call
COMPLETIONS_API_PARAMS = {
    "engine": COMPLETIONS_MODEL,
    "prompt": prompt,
    "max_tokens": 200,
    "temperature": 0,
    "stop": "#"
}

response = openai.Completion.create(
                **COMPLETIONS_API_PARAMS
            )

In [9]:
print("Prompt:\n" + prompt + "\n")
print("Response:\n" + response["choices"][0]["text"])

Prompt:
# read a CSV file using PySpark and print the schema
def 

Response:
read_csv_file(spark, file_name):
    df = spark.read.csv(file_name, header=True, inferSchema=True)
    df.printSchema()
    return df




### Low max_tokens with custom stop "\n" to limit response to one line example

In [10]:
COMPLETIONS_MODEL = "code-davinci-002"
prompt = "# read a CSV file using PySpark and print the schema\ndef "

# Set the parameter for the completions API call
COMPLETIONS_API_PARAMS = {
    "engine": COMPLETIONS_MODEL,
    "prompt": prompt,
    "max_tokens": 200,
    "temperature": 0,
    "stop": "\n"
}

response = openai.Completion.create(
                **COMPLETIONS_API_PARAMS
            )

In [11]:
print("Prompt:\n" + prompt + "\n")
print("Response:\n" + response["choices"][0]["text"])

Prompt:
# read a CSV file using PySpark and print the schema
def 

Response:
read_csv_file(spark, file_name):


# N

It defines how many completitions to generate for each prompt.

Keep N low as this will rapdily reduce your tokens quota.

NB: temperature has to be greater than 0 to avoid identical generations

In [35]:
COMPLETIONS_MODEL = "code-davinci-002"
prompt = "# read a CSV file using PySpark and print the schema\ndef "

n = 5

# Set the parameter for the completions API call
COMPLETIONS_API_PARAMS = {
    "engine": COMPLETIONS_MODEL,
    "prompt": prompt,
    "max_tokens": 200,
    "temperature": 0.2,
    "stop": "#",
    "n": n
}

response = openai.Completion.create(
                **COMPLETIONS_API_PARAMS
            )

In [36]:
print("Prompt:\n" + prompt + "\n")

for i in range(n):
    print("Response:\n" + response["choices"][i]["text"])

Prompt:
# read a CSV file using PySpark and print the schema
def 

Response:
read_csv_file(spark, filename):
    df = spark.read.csv(filename, header=True, inferSchema=True)
    df.printSchema()
    return df


Response:
read_csv(spark, filename):
    df = spark.read.csv(filename, header=True, inferSchema=True)
    df.printSchema()
    return df


Response:
read_csv_file(filename):
    df = spark.read.csv(filename, header=True, inferSchema=True)
    df.printSchema()
    return df


Response:
read_csv_file(file_name):
    df = spark.read.csv(file_name, header=True, inferSchema=True)
    df.printSchema()
    return df


Response:
read_csv_file(spark, file_path):
    df = spark.read.csv(file_path, header=True, inferSchema=True)
    df.printSchema()
    return df




# Explain code

In [6]:
# Explain what the previous function is doing: It

In [7]:
COMPLETIONS_MODEL = "code-davinci-002"
prompt = """
def construct_prompt(question, context_embeddings, df):
    most_relevant_document_sections = order_document_sections_by_query_similarity(question, context_embeddings)
    
    chosen_sections = []
    chosen_sections_len = 0
    chosen_sections_indexes = []
     
    for _, section_index in most_relevant_document_sections:
        document_section = df.loc[section_index]
        
        chosen_sections_len += document_section.tokens #+ separator_len
        if chosen_sections_len > MAX_SECTION_LEN:
            break
            
        chosen_sections.append(document_section.content.replace("\n", " "))
        chosen_sections_indexes.append(str(section_index))
            
    print(f"Selected {len(chosen_sections)} document sections:")
    print("\n".join(chosen_sections_indexes))
    
    header = "If you don't get the info from the embeddings, just say 'I don't know'."
    
    return header + "".join(chosen_sections) + "\n\n Q: " + question
""" + "\nExplain what the previous function is doing: It"

# Set the parameter for the completions API call
COMPLETIONS_API_PARAMS = {
    "engine": COMPLETIONS_MODEL,
    "prompt": prompt,
    "max_tokens": 200,
    "temperature": 0.2,
    "stop": "\n"
}

response = openai.Completion.create(
                **COMPLETIONS_API_PARAMS
            )

In [8]:
print("Prompt:\n" + prompt + "\n")
print("Response:\nIt" + response["choices"][0]["text"])

Prompt:

def construct_prompt(question, context_embeddings, df):
    most_relevant_document_sections = order_document_sections_by_query_similarity(question, context_embeddings)
    
    chosen_sections = []
    chosen_sections_len = 0
    chosen_sections_indexes = []
     
    for _, section_index in most_relevant_document_sections:
        document_section = df.loc[section_index]
        
        chosen_sections_len += document_section.tokens #+ separator_len
        if chosen_sections_len > MAX_SECTION_LEN:
            break
            
        chosen_sections.append(document_section.content.replace("
", " "))
        chosen_sections_indexes.append(str(section_index))
            
    print(f"Selected {len(chosen_sections)} document sections:")
    print("
".join(chosen_sections_indexes))
    
    header = "If you don't get the info from the embeddings, just say 'I don't know'."
    
    return header + "".join(chosen_sections) + "

 Q: " + question

Explain what the previous functi